In [1]:
import pandas as pd
import os
import shutil
import sqlite3

In [3]:
landing = '/workspaces/api-geral/GEO/CEP/dados/landing/'

leitor_landing = os.listdir(landing)
print(leitor_landing)

['resultado_concat_20240501_162447.txt']


In [36]:
from datetime import datetime
import os.path


timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
landing = '/workspaces/api-geral/GEO/CEP/dados/landing/'
word_key = 'resultado'

ultimo_timestamp = None
caminho_arquivo = None


for root, dirs, files in os.walk(landing):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.txt', '.csv'}:
                        print(f'file extension csv or txt, ok! extension: {file_extension}')
                    elif file_extension.lower() in {'.bin'}:
                        print('file extension binary, ok')
                        print(file_extension)
                    else:
                        print('extension error, not found')

if caminho_arquivo:
    df = pd.read_csv(caminho_arquivo, sep=';')
    file_bckp = f'{caminho_arquivo}'
    df_ccep = df['cep'].str.replace(r'-', '')
    df_cep =df['cep'].str.extract(r'(\d{5})-\d{3}')
    df_cep_compl = df['cep'].str.extract(r'(-\d{3})')
    df['ccep'] = df_ccep
    df['cep_loc'] = df_cep
    df['cep_compl'] = df_cep_compl
    df_rplc = df['cep_compl'].str.replace('-', '')
    df['ccep_compl'] = df_rplc
    columns = ['cep', 'cep_compl']
    df.drop(columns, inplace=True, axis=1)
    df['ccep'].astype(int)
    df['cep_loc'].astype(int)
    df['ccep_compl'].astype(int)
    
    
    os.system(f'cp {caminho_arquivo} {bckp_path}')
    print(f'file copy: {file_bckp}')
    display(df)
                           
                    
                        
                
                

True
file extension csv or txt, ok! extension: .txt
file copy: /workspaces/api-geral/GEO/CEP/dados/landing/resultado_concat_20240513_162447.txt


,logradouro,bairro,localidade,uf,ibge,gia,ddd,siafi,ccep,cep_loc,ccep_compl
0,Rua Mairiporã,Mantiqueira,Mairiporã,SP,3528502,4339,11,6671,07613580,07613,580
1,Rua JasminsJd Mairipora I,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605020,07605,020
2,Rua OrquideasJd Mairipora I,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605065,07605,065
3,Rua GirassolJd Mairipora II,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605225,07605,225
4,Rua das RosasJd Mairipora II,Rio Abaixo,Mairiporã,SP,3528502,4339,11,6671,07605200,07605,200
...,...,...,...,...,...,...,...,...,...,...,...
211,Avenida das Carnaubas,Olho D'Água,Mairiporã,SP,3528502,4339,11,6671,07601170,07601,170
212,Avenida das Camélias,Roseira,Mairiporã,SP,3528502,4339,11,6671,07619228,07619,228
213,Avenida dos Coqueiros,Barreiro,Mairiporã,SP,3528502,4339,11,6671,07611400,07611,400
214,Avenida Manoel Martins,Samambaia,Mairiporã,SP,3528502,4339,11,6671,07623025,07623,025


In [38]:
import json 
import os
from datetime import datetime
import os.path

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

bckp_path = '/workspaces/api-geral/GEO/CEP/ingestion/dados_bckp/'
layout = '/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/'
word_key = 'resultado_cep'


ultimo_timestamp = None
caminho_arquivo = None


for root, dirs, files in os.walk(layout):
    for nome_arquivo in files:
        if word_key in nome_arquivo:
            caminho_completo = os.path.join(root, nome_arquivo)
            timestamp = os.path.getmtime(caminho_completo)
            if ultimo_timestamp is None or timestamp > ultimo_timestamp:
                ultimo_timestamp = timestamp
                caminho_arquivo = caminho_completo
                checking = os.path.isfile(caminho_arquivo)
                if checking == True:
                    print(checking)
                else:
                    print('Error checking')
                if os.path.isfile(caminho_arquivo):
                    file_extension = os.path.splitext(caminho_arquivo)[1]
                    if file_extension.lower() in {'.json'}:
                        print(f'file extension json, ok! extension: {file_extension}')
                    else:
                        print('extension json error, not found')
                    print(caminho_arquivo)



True
file extension json, ok! extension: .json
/workspaces/api-geral/GEO/CEP/ingestion/dados_ing/layout/resultado_cep.json


In [ ]:
import os

file_path = "C:/folder/file.mp3"
if os.path.isfile(file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension.lower() in {'.mp3', '.flac', '.ogg'}:
        print("It's a music file")
    elif file_extension.lower() in {'.jpg', '.jpeg', '.png'}:
        print("It's an image file")

In [ ]:
                for check in caminho_arquivo:
                    ext = os.path.splitext(check)[-2].lower()
                    if ext == '.txt':
                        print(f'extension: {check}')
                    elif ext == '.csv':
                        print(f'extension: {check}')
                    else:
                        print(f'error extension format.')

In [ ]:
filepaths = ["/folder/soundfile.mp3", "folder1/folder/soundfile.flac"]

for fp in filepaths:
    # Split the extension from the path and normalise it to lowercase.
    ext = os.path.splitext(fp)[-1].lower()

    # Now we can simply use == to check for equality, no need for wildcards.
    if ext == ".mp3":
        print fp, "is an mp3!"
    elif ext == ".flac":
        print fp, "is a flac file!"
    else:
        print fp, "is an unknown file format."